In [ ]:
import nrrd
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
'''
# Define paths to the folder containing .nrrd files and the .xls file
nrrd_folder = "Train"
xls_file = "dataset_lung.xlsx"

# Load the Excel file
patient_data = pd.read_excel(xls_file)

# Example to load a full slice and nodule
full_slice_path = "pat1_fullslice.nrrd"
nodule_path = "pat1_nodule.nrrd"

# Load the NRRD files
full_slice_data, _ = nrrd.read(full_slice_path)
nodule_data, _ = nrrd.read(nodule_path)

# Display the full slice and nodule images
plt.subplot(1, 2, 1)
plt.title("Full Slice")
plt.imshow(full_slice_data, cmap='gray')

plt.subplot(1, 2, 2)
plt.title("Nodule")
plt.imshow(nodule_data, cmap='gray')

plt.show()
'''

In [ ]:
'''import cv2
import os
import numpy as np

# Paths
full_slice_folder = "full_slices/"      # Folder containing full slice images
nodule_folder = "nodules/"              # Folder containing nodule images (templates)
output_label_folder = "labels/"         # YOLOv5 label output folder
os.makedirs(output_label_folder, exist_ok=True)

# Function to perform template matching
def detect_nodule_and_save_labels(full_image_path, nodule_templates, output_label_path):
    image = cv2.imread(full_image_path, cv2.IMREAD_GRAYSCALE)
    image_height, image_width = image.shape[:2]
    bboxes = []  # List to store bounding boxes

    for template_path in nodule_templates:
        template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
        h, w = template.shape[:2]

        # Perform template matching
        result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
        threshold = 0.8  # Adjust threshold for confidence
        loc = np.where(result >= threshold)

        # Collect bounding boxes
        for pt in zip(*loc[::-1]):  # Switch columns and rows
            x, y = pt[0], pt[1]
            bbox = [x + w // 2, y + h // 2, w, h]  # Centered bounding box
            bboxes.append(bbox)

    # Normalize and save bounding boxes in YOLO format
    with open(output_label_path, 'w') as f:
        for bbox in bboxes:
            x_center = bbox[0] / image_width
            y_center = bbox[1] / image_height
            norm_width = bbox[2] / image_width
            norm_height = bbox[3] / image_height
            f.write(f"0 {x_center} {y_center} {norm_width} {norm_height}\n")  # Class 0 for nodule

# Run the detection
full_slice_images = [os.path.join(full_slice_folder, img) for img in os.listdir(full_slice_folder)]
nodule_templates = [os.path.join(nodule_folder, img) for img in os.listdir(nodule_folder)]

for full_image_path in full_slice_images:
    image_name = os.path.basename(full_image_path).replace('.png', '.txt').replace('.jpg', '.txt')
    output_label_path = os.path.join(output_label_folder, image_name)
    detect_nodule_and_save_labels(full_image_path, nodule_templates, output_label_path)'''


In [13]:
import pandas as pd

# Define paths to the folder containing .nrrd files and the .xls file
nrrd_folder = "Train"
excel_path = 'dataset_lung.xlsx'
df = pd.read_excel(excel_path)

# Display the Excel content
print(df.head())

            Full_slice            Nodule  TumorClass
0  pat1_fullslice.nrrd  pat1_nodule.nrrd           5
1  pat2_fullslice.nrrd  pat2_nodule.nrrd           5
2  pat3_fullslice.nrrd  pat3_nodule.nrrd           2
3  pat4_fullslice.nrrd  pat4_nodule.nrrd           5
4  pat5_fullslice.nrrd  pat5_nodule.nrrd           4


In [15]:
import nrrd
import cv2
import numpy as np
import os

# Folder paths
images_dir = './images'
labels_dir = './labels'

os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Iterate through Excel rows
for index, row in df.iterrows():
    full_slice_path = row['Full_slice']     # Full slice filename
    nodule_path = row['Nodule']             # Nodule filename
    tumor_class = int(row['TumorClass'])   # Tumor class (class_id)

    # print(os.path.join(nrrd_folder, full_slice_path))
    
    # Load NRRD files
    full_slice_data, _ = nrrd.read(os.path.join(nrrd_folder, full_slice_path))
    nodule_data, _ = nrrd.read(os.path.join(nrrd_folder, nodule_path))

    # Normalize for OpenCV compatibility
    full_slice = (full_slice_data * 255).astype(np.uint8)
    nodule_template = (nodule_data * 255).astype(np.uint8)
    
    # Template Matching to find bounding box
    result = cv2.matchTemplate(full_slice, nodule_template, cv2.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv2.minMaxLoc(result)

    # Bounding box coordinates
    h, w = nodule_template.shape
    x, y = max_loc
    bbox = (x, y, w, h)

    # Normalize bounding box for YOLO format
    img_height, img_width = full_slice.shape
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_width = w / img_width
    norm_height = h / img_height

    # Save the full slice image
    image_output_path = os.path.join(images_dir, f"pat{index+1}.jpg")
    cv2.imwrite(image_output_path, full_slice)

    # Save the label in YOLO format (use tumor_class as class_id)
    label_output_path = os.path.join(labels_dir, f"pat{index+1}.txt")
    with open(label_output_path, 'w') as f:
        f.write(f"{tumor_class} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n")

print("Dataset preparation complete with tumor class labels!")


Dataset preparation complete with tumor class labels!
